In [2]:
import pandas as pd
import numpy as np

### Cargar datos

In [2]:
listings = pd.read_csv("Beijing Airbnb Data(clean)/listings_clean.csv")

In [49]:
calendar = pd.read_csv("Beijing Airbnb Data(clean)/calendar_detail.csv")

In [4]:
reviews = pd.read_csv("Beijing Airbnb Data(clean)/reviews_detail.csv")

In [5]:
print(listings.shape,reviews.shape,calendar.shape)

(28448, 15) (202099, 5) (10384980, 7)


#### Pre-procesamiento listings

In [6]:
listings.head(5)

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,44054,Modern and Comfortable Living in CBD,192875,East Apartments,朝阳区,39.89503,116.45163,Entire home/apt,792,1,89,2019-03-04,0.85,9,341
1,100213,The Great Wall Box Deluxe Suite A团园长城小院东院套房,527062,Joe,密云县,40.68434,117.17231,Private room,1201,1,2,2017-10-08,0.10,4,0
2,128496,Heart of Beijing: House with View 2,467520,Cindy,东城区,39.93213,116.42200,Entire home/apt,389,3,259,2019-02-05,2.70,1,93
3,161902,cozy studio in center of Beijing,707535,Robert,东城区,39.93357,116.43577,Entire home/apt,376,1,26,2016-12-03,0.28,5,290
4,162144,"nice studio near subway, sleep 4",707535,Robert,朝阳区,39.93668,116.43798,Entire home/apt,537,1,37,2018-08-01,0.40,5,352


In [18]:
listings.isna().sum()

id                  0
name                0
host_id             0
host_name           0
room_type           0
price               0
minimum_nights      0
availability_365    0
dtype: int64

In [11]:
listings['last_review']=listings['last_review'].replace(np.nan, 'No reviews', regex=True)

In [12]:
listings.isna().sum()

id                                0
name                              0
host_id                           0
host_name                         0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

In [13]:
loc=listings[['id','latitude','longitude']]

In [15]:
loc.to_csv('db_final/location.csv',index=False)

In [16]:
listings=listings.drop(['neighbourhood','number_of_reviews','last_review','reviews_per_month',
                        'calculated_host_listings_count','latitude','longitude'], axis=1)

In [17]:
listings.to_csv('listings_final.csv',index=False) #para traducir

In [40]:
listings = pd.read_csv("table_listing.csv") #traducido

In [41]:
listings.head(5)

,id,name,host_id,host_name,room_type,price,minimum_nights,availability_365,Unnamed: 8,name_en,host_name_en
0,44054,Modern and Comfortable Living in CBD,192875,East Apartments,Entire home/apt,792,1,341,NaN,Modern and Comfortable Living in CBD,East Apartments
1,100213,The Great Wall Box Deluxe Suite A团园长城小院东院套房,527062,Joe,Private room,1201,1,0,NaN,The Great Wall Box Deluxe Suite A Group Garden...,Joe
2,128496,Heart of Beijing: House with View 2,467520,Cindy,Entire home/apt,389,3,93,NaN,Heart of Beijing: House with View 2,Cindy
3,161902,cozy studio in center of Beijing,707535,Robert,Entire home/apt,376,1,290,NaN,cozy studio in center of Beijing,Robert
4,162144,"nice studio near subway, sleep 4",707535,Robert,Entire home/apt,537,1,352,NaN,"nice studio near subway, sleep 4",Robert


In [42]:
listings=listings.drop(['name','host_name', 'Unnamed: 8'], axis=1)
listings.rename(columns={'name_en': 'name', 'host_name_en':'host_name'}, inplace=True)

In [43]:
listings=listings[['id','name','host_id','host_name','room_type','price','minimum_nights','availability_365']]

In [44]:
listings['name']=listings['name'].str.replace(r'\n', ' ')
listings['name']=listings['name'].str.replace('\s+', ' ')
listings['name']=listings['name'].str.replace('[^0-9a-zA-Z.,\s]', ' ')

<ipython-input-44-d2caa83fa581>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  listings['name']=listings['name'].str.replace(r'\n', ' ')
<ipython-input-44-d2caa83fa581>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  listings['name']=listings['name'].str.replace('\s+', ' ')
<ipython-input-44-d2caa83fa581>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  listings['name']=listings['name'].str.replace('[^0-9a-zA-Z.,\s]', ' ')


In [45]:
listings.head(5)

,id,name,host_id,host_name,room_type,price,minimum_nights,availability_365
0,44054,Modern and Comfortable Living in CBD,192875,East Apartments,Entire home/apt,792,1,341
1,100213,The Great Wall Box Deluxe Suite A Group Garden...,527062,Joe,Private room,1201,1,0
2,128496,Heart of Beijing House with View 2,467520,Cindy,Entire home/apt,389,3,93
3,161902,cozy studio in center of Beijing,707535,Robert,Entire home/apt,376,1,290
4,162144,"nice studio near subway, sleep 4",707535,Robert,Entire home/apt,537,1,352


In [46]:
listings.to_csv('db_final/listings.csv',index=False)

#### Pre-procesamiento reviews

In [31]:
for x in reviews.listing_id.array:
    if x not in listings.id.array:
        reviews.drop(reviews.loc[reviews['listing_id']==x].index, inplace=True)

In [32]:
reviews.shape

(202018, 5)

In [49]:
reviews=reviews.drop(columns=['id'], axis=1)

In [50]:
len(reviews.listing_id.unique())

17293

In [51]:
n = reviews.shape[0]
reviews_sample = reviews.groupby('listing_id', group_keys=False).apply(lambda x: x.sample(int(np.ceil(50000*len(x)/len(reviews))))).sample(frac=1).reset_index(drop=True)



In [53]:
len(reviews.listing_id.unique())

17293

In [54]:
reviews_sample.head(5)

,listing_id,date,reviewer_name,comments
0,24635576,24-03-19,源媛,"这个地方位置特别好,闹中取静｡特别适合带老人孩子来,里外间的方型,走到南锣鼓巷地铁站也就几分..."
1,22512229,20-01-19,普,不错 很值的 很干净整洁下次还会去
2,20201819,04-04-19,远凝,The host canceled this reservation the day bef...
3,13901242,29-08-16,向阳,Nice party place. Nice design.\r\nDont hesitat...
4,17898612,07-10-17,Yuka,"小姐姐很nice,so easy-going ,是一位很漂亮的小姐姐呢~ 记得有天我回家晚..."


In [57]:
reviews_sample.shape

(58381, 4)

In [17]:
reviews_sample.to_csv('reviews_final.csv',index=False) #a traducir

NameError: name 'reviews_sample' is not defined

In [33]:
reviews = pd.read_csv("reviews_final2.csv") #traducido

In [34]:
reviews.head(5)

,listing_id,date,reviewer_name,comments,Unnamed: 4,Unnamed: 5
0,24635576,24-03-19,源媛,"这个地方位置特别好,闹中取静｡特别适合带老人孩子来,里外间的方型,走到南锣鼓巷地铁站也就几分...",Source Yuan,"This place is particularly good location, quie..."
1,22512229,20-01-19,普,不错 很值的 很干净整洁下次还会去,general,Very good value very clean and tidy will go next
2,20201819,04-04-19,远凝,The host canceled this reservation the day bef...,Far condensate,The host canceled this reservation the day bef...
3,13901242,29-08-16,向阳,Nice party place. Nice design.\nDont hesitate ...,Sunningdale,Nice party place. Nice design.\nDont hesitate ...
4,17898612,07-10-17,Yuka,"小姐姐很nice,so easy-going ,是一位很漂亮的小姐姐呢~ 记得有天我回家晚...",Yuka,"Little sister is very nice, so easy-going, is ..."


In [35]:
reviews=reviews.drop(['reviewer_name','comments'], axis=1)
reviews.rename(columns={'Unnamed: 4': 'reviewer_name', 'Unnamed: 5':'comments'}, inplace=True)

In [36]:
reviews.head(5)

,listing_id,date,reviewer_name,comments
0,24635576,24-03-19,Source Yuan,"This place is particularly good location, quie..."
1,22512229,20-01-19,general,Very good value very clean and tidy will go next
2,20201819,04-04-19,Far condensate,The host canceled this reservation the day bef...
3,13901242,29-08-16,Sunningdale,Nice party place. Nice design.\nDont hesitate ...
4,17898612,07-10-17,Yuka,"Little sister is very nice, so easy-going, is ..."


In [37]:
reviews['comments']=reviews['comments'].str.replace(r'\n', ' ')
reviews['comments']=reviews['comments'].str.replace('\s+', ' ')
reviews['comments']=reviews['comments'].str.replace('[^0-9a-zA-Z.,\s]', ' ')
reviews['reviewer_name']=reviews['reviewer_name'].str.replace(r'\n', ' ')
reviews['reviewer_name']=reviews['reviewer_name'].str.replace('\s+', ' ')
reviews['reviewer_name']=reviews['reviewer_name'].str.replace('[^0-9a-zA-Z.,\s]', ' ')

<ipython-input-37-4f1c0b100152>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  reviews['comments']=reviews['comments'].str.replace(r'\n', ' ')
<ipython-input-37-4f1c0b100152>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  reviews['comments']=reviews['comments'].str.replace('\s+', ' ')
<ipython-input-37-4f1c0b100152>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  reviews['comments']=reviews['comments'].str.replace('[^0-9a-zA-Z.,\s]', ' ')
<ipython-input-37-4f1c0b100152>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  reviews['reviewer_name']=reviews['reviewer_name'].str.replace(r'\n', ' ')
<ipython-input-37-4f1c0b100152>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  reviews['reviewer_name']=reviews['reviewer_name'].str.replace('\s+

In [38]:
reviews.head(5)

,listing_id,date,reviewer_name,comments
0,24635576,24-03-19,Source Yuan,"This place is particularly good location, quie..."
1,22512229,20-01-19,general,Very good value very clean and tidy will go next
2,20201819,04-04-19,Far condensate,The host canceled this reservation the day bef...
3,13901242,29-08-16,Sunningdale,Nice party place. Nice design. Dont hesitate t...
4,17898612,07-10-17,Yuka,"Little sister is very nice, so easy going, is ..."


In [48]:
reviews.isna().sum()

listing_id       0
date             0
reviewer_name    0
comments         0
dtype: int64

In [39]:
reviews.shape

(58381, 4)

In [47]:
reviews.to_csv('db_final/reviews.csv',index=False)

#### Pre-procesamiento calendar

In [51]:
calendar.head(5)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,1165040,2019-04-17,f,$511.00,$511.00,1.0,1125.0
1,1165040,2019-04-18,t,$511.00,$511.00,1.0,1125.0
2,1165040,2019-04-19,t,$511.00,$511.00,1.0,1125.0
3,1165040,2019-04-20,t,$511.00,$511.00,1.0,1125.0
4,1165040,2019-04-21,t,$511.00,$511.00,1.0,1125.0


In [52]:
len(calendar.listing_id.unique())

28452

In [54]:
sample_calendar = calendar.groupby('listing_id', group_keys=False).apply(lambda x: x.sample(int(np.ceil(50000*len(x)/len(calendar))))).sample(frac=1).reset_index(drop=True)
sample_calendar['price']=sample_calendar['price'].str.replace(r',', '')
sample_calendar['price']=sample_calendar['price'].str.replace(r'$', '')
sample_calendar['adjusted_price']=sample_calendar['adjusted_price'].str.replace(r',', '')
sample_calendar['adjusted_price']=sample_calendar['adjusted_price'].str.replace(r'$', '')

<ipython-input-54-f72a22eee1d9>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  sample_calendar['price']=sample_calendar['price'].str.replace(r'$', '')
<ipython-input-54-f72a22eee1d9>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  sample_calendar['adjusted_price']=sample_calendar['adjusted_price'].str.replace(r'$', '')


In [55]:
len(sample_calendar.listing_id.unique())

28452

In [56]:
sample_calendar.reset_index(drop=True, inplace=True)
sample_calendar.dropna(inplace=True)

In [57]:
sample_calendar.shape

(56902, 7)

In [58]:
sample_calendar.to_csv('db_final/calendar.csv',index=False)